In [1]:
import matplotlib
matplotlib.use("TkAgg") #Makes imshow work on mac
from matplotlib import pyplot as plt
from Model import Model, draw_boxes
import cozmo
import queue
import time
import threading
import numpy as np
import sys
import pandas as pd
import base64
import CS481Dataset as CS

import datetime
import csv
import PIL
from PIL import Image

import LanguageModel as LM
import pickle

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
def detectImages():
    global data
    confidence = 0 # imgCount
    print('Detect Images started') # 'Detect Images started'
    
    ds = []
    
    while(True):
        if(not imageQueue.empty()):
            try:

                pilImg = imageQueue.get()
                box, imgResized = model.object_detect(pilImg.copy())
                cleanImg = imgResized.copy();
                draw_boxes(box, imgResized, (model.size, model.size))

                imgWithBox = np.array(imgResized) 
                imgWithBox = imgWithBox[:, :, ::-1].copy()
                box = box[0]
                formattedBox = (box[1], box[0], box[3], box[2]) # coordinates need to be corrected for crop
                imgBox = cleanImg.crop(formattedBox)
                plt.imshow(imgBox)
                plt.pause(0.001) # imshow needs time to plot the image. Need this to display the image

                print('???')
                label = input()
#                 if response.strip() == 'y':
#                 CS481Dataset.encodeAndSaveImage(data, pilImg, label, formattedBox) 
                
                words = label.split()
    
                imgNumpy = np.asarray(pilImg)
                imgStr = base64.b64encode(imgNumpy)
                imgStr = str(imgStr)[2:-1]
              
                dimensions = list(pilImg.size)[::-1] + [3]
                time = str(datetime.datetime.now()) #[10:]
                
                for w in words:
                    row = (imgStr, w, dimensions, formattedBox, time)
                    ds.append(row)
    
                confidence+=1
                print(label + '\n')
                print('img count: ' + str(confidence) + '\n')
                            
                print('save?')
                response = input()
                if response.strip() == 'y':
                    
                    datafr = pd.DataFrame(ds, columns=['img','label','dimensions','box_coord','time'])
                    datafr.to_csv(dataset, index=False)
#                     data.to_csv(dataset, index=False)
                    print('saved' + '\n')
                else:
                    continue

            except queue.Empty:
                pass

In [3]:
def encode_save_image(df: pd.DataFrame, img: PIL.Image, label: str, boundingBox):
    imgNumpy = np.asarray(img)
    imgStr = base64.b64encode(imgNumpy)

    time = str(datetime.datetime.now())[10:]
    
    df.loc[len(df)] = [str(imgStr)[2:-1], label, list(img.size)[::-1] + [3], boundingBox, time]

In [4]:
def handle_image(evt, obj=None, tap_count=None,  **kwargs):
    try:
        if(imageQueue.empty()):
            imageQueue.put_nowait(evt.image)
    except queue.Full:
        pass

In [5]:
def cozmo_program(robot: cozmo.robot.Robot):
    robot.camera.color_image_enabled = True
    robot.add_event_handler(cozmo.camera.EvtNewRawCameraImage, handle_image)
    robot.set_lift_height(1.0)
    print("Added event handler")
    while True:
        time.sleep(0.1)

In [7]:
model = Model(path='../f18/data/coco2014', jpegs='../f18/train2014', bb_csv='../f18/data/coco2014/tmp/bb.csv')
imageQueue = queue.Queue(maxsize=1)

# date = str(datetime.datetime.now())[:10]
# dataset = 'dataset.csv'
dataset = 'explore_history.csv'

# row = ['img','label','dimensions','time']
# with open(dataset, 'w') as writeFile:
#     writer = csv.writer(writeFile)
#     writer.writerow(row)

# data = pd.DataFrame({'img':[''],
#                   'label':[''],
#                   'dimensions':[''],
#                      'box_coord':[''],
#                   'time':['']})

data = pd.read_csv(dataset)
data.head()

,img,label,dimensions,box_coord,time
0,DAwMDAwMDQ0NDg4ODg4ODw8PEBAQEBAQDAwMDQ0NDw8PEB...,blocks,"[240, 320, 3]","(0.08415357, 36.06744, 211.52437, 196.10556)",2019-07-30 16:01:34.128181
1,DAwMDAwMDQ0NDg4ODg4ODw8PEBAQEBAQDg4ODg4ODg4ODg...,black and white,"[240, 320, 3]","(11.060257, 27.61384, 229.36584, 195.69952)",2019-07-30 16:01:42.700762
2,Cw0ICw0ICw0ICw0IDA0IDA0IDwsIDwsIEAsIEAsIEwkIEw...,purple,"[240, 320, 3]","(53.525486, 51.073635, 204.68407, 151.20416)",2019-07-30 16:02:02.150410
3,Dg8KDg8KDg8KDg8KDw4KDw4KDw4KDw4KEQ0KEQ0KEQ0KEQ...,dark blue Z,"[240, 320, 3]","(55.54578, 41.274704, 209.2253, 142.09975)",2019-07-30 16:02:09.358059


In [8]:
threading.Thread(target=detectImages).start()

cozmo.run_program(cozmo_program, use_viewer=True)

Detect Images started


2019-07-30 16:53:48,547 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2019-07-30 16:53:48,605 cozmo.general INFO     Found robot id=1
2019-07-30 16:53:48,886 cozmo.general INFO     Connected to Android device serial=03160355293b2004
2019-07-30 16:53:48,944 cozmo.general INFO     Robot id=1 serial=024086a7 initialized OK
Exception in Tkinter callback
/home/david/GIT/cs481-senior-design/s19/Model.py:87: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  scipy.misc.imsave(tempFile, img_resized)
/home/david/anaconda3/envs/oldfastai/lib/python3.6/site-packages/scipy/misc/pilutil.py:217: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, channel_axis=2)
/home/david/anaconda3/envs/o

Added event handler
???
skip
skip

img count: 1

save?
n
???
skip
skip

img count: 2

save?
n
???
blue Z
blue Z

img count: 3

save?
y
saved

???


2019-07-30 16:54:25,515 cozmo.general INFO     Shutting down connection
2019-07-30 16:54:25,517 cozmo.general INFO     Android serial=03160355293b2004 disconnected.
2019-07-30 16:54:25,519 cozmo.general INFO     Exit requested by user


## Build Language Model

### ... append to Language Model???

In [2]:
# datafile = 'dataset.csv'
datafile = 'explore_history.csv'

In [4]:
datafile

'explore_history.csv'

In [5]:
#Load data
print('Loading dataset')
dataOrig = CS.loadFromCSV(datafile)
LM_data = LM.prepareDataset(dataOrig)

Loading dataset


FileNotFoundError: [Errno 2] File b'explore_history.csv' does not exist: b'explore_history.csv'

In [14]:
LM_data

,img,word,box_coord,time,annotation,imgFeatures
0,"[[[12, 12, 12], [12, 12, 12], [13, 13, 13], [1...",blocks,"(0.08415357, 36.06744, 211.52437, 196.10556)",2019-07-30 16:01:34.128181,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.138063, 0.0, ..."
1,"[[[12, 12, 12], [12, 12, 12], [13, 13, 13], [1...",black and white,"(11.060257, 27.61384, 229.36584, 195.69952)",2019-07-30 16:01:42.700762,1,"[0.0, 0.0, 0.0, 0.28849867, 0.0, 0.0, 2.051689..."
2,"[[[11, 13, 8], [11, 13, 8], [11, 13, 8], [11, ...",purple,"(53.525486, 51.073635, 204.68407, 151.20416)",2019-07-30 16:02:02.150410,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9238289, 0.0,..."
3,"[[[14, 15, 10], [14, 15, 10], [14, 15, 10], [1...",dark blue Z,"(55.54578, 41.274704, 209.2253, 142.09975)",2019-07-30 16:02:09.358059,1,"[0.0, 0.0, 0.0, 0.0, 0.36562327, 0.0, 0.984727..."
4,"[[[12, 12, 12], [12, 12, 12], [13, 13, 13], [1...",dark blue Z,"(0.08415357, 36.06744, 211.52437, 196.10556)",2019-07-30 16:01:34.128181,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.138063, 0.0, ..."
5,"[[[14, 15, 10], [14, 15, 10], [14, 15, 10], [1...",blocks,"(55.54578, 41.274704, 209.2253, 142.09975)",2019-07-30 16:02:09.358059,0,"[0.0, 0.0, 0.0, 0.0, 0.36562327, 0.0, 0.984727..."
6,"[[[12, 12, 12], [12, 12, 12], [13, 13, 13], [1...",black and white,"(0.08415357, 36.06744, 211.52437, 196.10556)",2019-07-30 16:01:34.128181,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.138063, 0.0, ..."
7,"[[[14, 15, 10], [14, 15, 10], [14, 15, 10], [1...",purple,"(55.54578, 41.274704, 209.2253, 142.09975)",2019-07-30 16:02:09.358059,0,"[0.0, 0.0, 0.0, 0.0, 0.36562327, 0.0, 0.984727..."


In [ ]:
LM_data.to_pickle('explore_LM.pkl')

In [ ]:
#Split Data
train, test = train_test_split(data, test_size=0.2)

In [ ]:
#Create model
print('Adding images to model')
langMod = LM.LanguageModel()
langMod.addImages(train)
print('Training model')
langMod.train()